<a href="https://colab.research.google.com/github/dudns1234/Web-Scraping/blob/master/03_HTML_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
html_str = """
<html>
  <head>
    <title>안녕하세요</title>
  </head>
  <body>
    <div id="container">
      <p class='p1'>hello</p>
      <p>Bye</p>
    </div>
  </body>
</html>"""

# html_str = """
# <html>
#   <head>
#     <title>안녕하세요</title>
#   </head>
#   <body>
#     <div id="container">
#       <p class='p1'>hello</p>
#       <p>Bye</p>
#       <p class='p2'>ya</p>
#     </div>
#   </body>
# </html>"""

html 문자열을 파이썬에서 다뤄야 한다.
- HTML 문자열을 엘리먼트화 해서 파이썬에서 **객체**로 다룰 수 있어야 한다.
  - `BeautifulSoup`을 이용해서 HTML 문자열을 엘리먼트 객체로 바꿔줄 수 있다!

In [ ]:
from bs4 import BeautifulSoup # 이상한 나라의 앨리스

In [ ]:
soup = BeautifulSoup(html_str, 'html.parser')
soup


<html>
<head>
<title>안녕하세요</title>
</head>
<body>
<div id="container">
<p class="p1">hello</p>
<p>Bye</p>
</div>
</body>
</html>

In [ ]:
type(soup)

bs4.BeautifulSoup

# `find`를 이용하기
- `find("태그명", "속성 값을 딕셔너리")` : 한 개의 엘리먼트만 찾기
- `find_all("태그명", "속성 값 딕셔너리")` : 여러 개 찾기(리스트)

In [ ]:
# hello와 ya가져오기 -> find_all 이기 때문에 리스트 형태임. 그래서 .text로는 불가함.
soup.find_all("p")[::2]

[<p class="p1">hello</p>]

In [ ]:
# div 중에 id가 container인 것 찾기
soup.find("div", {"id" : "container"})

<div id="container">
<p class="p1">hello</p>
<p>Bye</p>
</div>

In [ ]:
# p 중에 class가 p1인 것 찾기
soup.find("p", {"class": "p1"})

<p class="p1">hello</p>

In [ ]:
# 모든 p 찾기
soup.find_all("p")

[<p class="p1">hello</p>, <p>Bye</p>]

In [ ]:
# 텍스트(text) 추출하기
soup.find("p", {"class" : "p1"}).text

'hello'

In [ ]:
# 여러 개의 엘리먼트 중 제일 먼저 나오는 엘리먼트 1개만 나온다.
soup.find("p")

<p class="p1">hello</p>

In [ ]:
# 제일 마지막 p 태그 꺼내오기
soup.find_all("p")[-1]

<p>Bye</p>

특수한 상황에서만 사용
- `nextSibling` : 바로 밑에 형제

In [ ]:
p_p1 = soup.find("p", {"class": "p1"})
p_p1

<p class="p1">hello</p>

In [ ]:
p_p1.nextSibling

'\n'

In [ ]:
p_p1.nextSibling.nextSibling

<p>Bye</p>

## 선택자(selector)를 사용해서 추출

In [ ]:
soup


<html>
<head>
<title>안녕하세요</title>
</head>
<body>
<div id="container">
<p class="p1">hello</p>
<p>Bye</p>
</div>
</body>
</html>

- `soup.select(선택자)` : 선택자에 의해 엘리먼트를 **여러 개** 선택
- `soup.select_one(선택자)` : 선택자에 의해 엘리먼트를 한 개만 선택


In [ ]:
soup.select_one("#container")

<div id="container">
<p class="p1">hello</p>
<p>Bye</p>
</div>

In [ ]:
soup.select_one("#container > .p1").text

'hello'

In [ ]:
soup.select("p")

[<p class="p1">hello</p>, <p>Bye</p>]

## 속성 값 추출

In [ ]:
soup.select_one(".p1")["class"]

['p1']

In [ ]:
soup.select_one(".p1").get("class")

['p1']

# 네이버 환율 정보 가져오기
- 웹에 접속을 해야 하기 때문에 `requests` 모듈 활용
- html문서를 가져오고 Element 객체화를 시키기 위해서 BeautifulSoup 필요

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
URL = "https://finance.naver.com/marketindex/"

In [ ]:
response = requests.get(URL)
page = response.content # 접속한 웹 사이트의 html 코드를 가져오기
page

In [ ]:
soup = BeautifulSoup(page, 'html.parser')
soup

In [ ]:
exchange_list = soup.select_one("#exchangeList")
exchange_list

In [ ]:
fin_list = exchange_list.find_all("li") # exchange_list.select("li")
fin_list

In [ ]:
sample_li = fin_list[0]
sample_li

In [ ]:
c_name = sample_li.find("h3", {"class":"h_lst"}).text
c_name

'미국 USD'

In [ ]:
c_name = sample_li.select_one("h3.h_lst").text
c_name

'미국 USD'

In [ ]:
exchange_rate = sample_li.select_one("span.value").text
exchange_rate

'1,349.50'

In [ ]:
change = sample_li.select_one("span.change").text
change

'4.50'

In [ ]:
updown = sample_li.select("span.blind")
updown[-1].text

'상승'

In [ ]:
updown = sample_li.select_one("span.change").nextSibling.nextSibling.text
updown

'상승'

In [ ]:
c_name_list = []
exchange_rate_list = []
change_list = []
updown_list = []

for fin in fin_list:
  c_name = fin.select_one("h3.h_lst").text
  exchange_rate = fin.select_one("span.value").text
  change = fin.select_one("span.change").text
  updown = fin.select_one("span.change").nextSibling.nextSibling.text

  print(c_name, exchange_rate, change, updown)

  c_name_list.append(c_name)
  exchange_rate_list.append(exchange_rate)
  change_list.append(change)
  updown_list.append(updown)

미국 USD 1,349.50 4.50 상승
일본 JPY(100엔) 900.99 2.08 상승
유럽연합 EUR 1,422.24 2.32 상승
중국 CNY 184.65 0.52 상승


In [ ]:
fin_datas = {
    "국가": c_name_list,
    "환율": exchange_rate_list,
    "변동": change_list,
    "등락": updown_list
}
fin_datas

{'국가': ['미국 USD', '일본 JPY(100엔)', '유럽연합 EUR', '중국 CNY'],
 '환율': ['1,349.50', '900.99', '1,422.24', '184.65'],
 '변동': ['4.50', '2.08', '2.32', '0.52'],
 '등락': ['상승', '상승', '상승', '상승']}

In [ ]:
import pandas as pd

df_finance = pd.DataFrame(fin_datas)
df_finance.to_csv("fin_2220415.csv")

In [ ]:
sample_li.select_one("span.blindasfdkdljks") # 선택자를 사용해서 선택을 했는데, 존재하지 않는 경우 : None으로 처리

In [ ]:
# python boolean 타입의 특징
a = None

# a에 값이 존재하면 True, 존재하지 않으면 False
# 0이라는 값의 의미는 존재하지 않는다는 정수
if a :
  print("hello")
else:
  print("bye")

bye


In [ ]:
a = sample_li.select_one("span.blind")
print(a)
if a :
  print("hello")
else:
  print("bye")

<span class="blind">미국 USD</span>
hello


# 무신사 반팔티 크롤링

In [ ]:
URL = "https://www.musinsa.com/category/001001"

res = requests.get(URL)
res

<Response [200]>

In [ ]:
res.content

In [ ]:
page = res.content
soup = BeautifulSoup(page, 'html.parser')
soup

In [ ]:
product_list = soup.select("#searchList > li")
len(product_list)

90

테스트!!

In [ ]:
sample_item1 = product_list[0]
sample_item1

<li class="li_box" data-no="2086653">
<!-- 상단 좌측 라벨 -->
<!-- 상단 우측 라벨 -->
<div class="box-icon-right">
<span class="icon-reverse icon-box-musinsa">무신사 단독</span>
</div>
<div class="icon_group">
<ul>
<li class="icon_man sight_out" title="남성">남성</li>
</ul>
</div>
<div class="li_inner">
<div class="list_img">
<a class="img-block" href="//www.musinsa.com/app/goods/2086653" name="goods_link" title="레이어드 크루 넥 반팔 티셔츠_일반 기장 [화이트]">
<img alt="무신사 스탠다드(MUSINSA STANDARD) 레이어드 크루 넥 반팔 티셔츠_일반 기장 [화이트]" class="lazyload lazy" data-original="https://image.msscdn.net/images/goods_img/20210825/2086653/2086653_1_125.jpg" src="//image.msscdn.net/images/no_image_125.png">
</img></a>
</div>
<div class="article_info">
<p class="item_title"><a href="//www.musinsa.com/brands/musinsastandard">무신사 스탠다드</a></p>
<p class="list_info">
<a href="//www.musinsa.com/app/goods/2086653" name="goods_link" title="레이어드 크루 넥 반팔 티셔츠_일반 기장 [화이트]">
<strong class="txt_reserve"></strong>
                  레이어드 크루 넥 반팔 티셔츠_일반 기장 [화이

In [ ]:
# image url 가져오기
sample_item1.select_one("img.lazyload")["data-original"]

'https://image.msscdn.net/images/goods_img/20210825/2086653/2086653_1_125.jpg'

In [ ]:
# 브랜드 가져오기
sample_item1.select_one("p.item_title > a").text

'무신사 스탠다드'

In [ ]:
# 상품명 가져오기
sample_item1.select_one("p.list_info").text.strip()

'레이어드 크루 넥 반팔 티셔츠_일반 기장 [화이트]'

In [ ]:
# 가격 가져오기
int(sample_item1.select_one("p.price").text.strip().replace(",", "").replace("원", ""))

16900

In [ ]:
# 별점 개수 가져오기
int(sample_item1.select_one("span.count").text.replace(",", ""))

47226

In [ ]:
sample_item2 = product_list[1]
sample_item2

<li class="li_box" data-no="2690691">
<!-- 상단 좌측 라벨 -->
<!-- 상단 우측 라벨 -->
<div class="icon_group">
<ul>
<li class="icon_man sight_out" title="남성">남성</li>
</ul>
</div>
<div class="li_inner">
<div class="list_img">
<a class="img-block" href="//www.musinsa.com/app/goods/2690691" name="goods_link" title="무지 머슬핏 반팔 13컬러">
<img alt="허그본(HUGVONE) 무지 머슬핏 반팔 13컬러" class="lazyload lazy" data-original="https://image.msscdn.net/images/goods_img/20220801/2690691/2690691_1_125.jpg" src="//image.msscdn.net/images/no_image_125.png">
</img></a>
</div>
<div class="article_info">
<p class="item_title"><a href="//www.musinsa.com/brands/hugvone">허그본</a></p>
<p class="list_info">
<a href="//www.musinsa.com/app/goods/2690691" name="goods_link" title="무지 머슬핏 반팔 13컬러">
<strong class="txt_reserve"></strong>
                  무지 머슬핏 반팔 13컬러
                </a>
</p>
<p class="price">
                
                22,000원
              </p>
<p class="mu-icon mu-icon-coupon">
<span class="txt_discount_price">-2

In [ ]:
# 나쁜 방법은 아니에요!!
sample_item2.select_one("p.price").text.strip().split()[1]

IndexError: ignored

In [ ]:
sample_item2.select_one("p.price > del").nextSibling.strip().replace(",", "").replace("원", "")

In [ ]:
product_img_url_list = []
product_brand_list = []
product_name_list = []
product_price_list = []
product_star_count_list = []

for product in product_list:
  img_url = product.select_one("img.lazyload")['data-original']

  # 브랜드
  if len(product.select("p.item_title")) > 1:
    product_title = product.select("p.item_title")[-1].text
  else:
    product_title = product.select_one("p.item_title").text

  # 상품명
  product_name = product.select_one("p.list_info").text.strip()

  # 가격
  price_del_elem = product.select_one("p.price").find("del")

  # <del> 태그가 있으면..
  if price_del_elem:
    product_price = price_del_elem.nextSibling.strip()
  else: # <del> 태그가 없으면..
    product_price = product.select_one("p.price").text.strip()

  product_price = int(product_price.replace(",", "").replace("원", ""))

  # 별점 등록 수
  star_count = int(product.select_one("p.point > span.count").text.replace(",", ""))

  product_img_url_list.append(img_url)
  product_brand_list.append(product_title)
  product_name_list.append(product_name)
  product_price_list.append(product_price)
  product_star_count_list.append(star_count)

In [ ]:
import pandas as pd
datas = {
    "상품이미지URL": product_img_url_list,
    "상품브랜드": product_brand_list,
    "상품명": product_name_list,
    "상품가격": product_price_list,
    "상품별점등록수": product_star_count_list
}

df = pd.DataFrame(datas)
df.head()

In [ ]:
df.groupby("상품브랜드").mean()

In [ ]:
df.tail()

In [ ]:
sample_item21 = product_list[21]
if len(sample_item21.select("p.item_title")) > 1:
  print(sample_item21.select("p.item_title")[-1])
else:
  print(sample_item21.select_one("p.item_title"))

In [ ]:
sample_item1

In [ ]:
sample_item1.select_one("p.txt_cnt_like")

In [ ]:
import json

API_URL = "https://like.musinsa.com/like/api/v1/liketypes/goods/counts"
params = {
    "relationIds":[2086653] # payload 탭에서 확인!
}

headers = {
    "content-type": "application/json",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"
}

response = requests.post(
    API_URL,
    data= json.dumps(params), # json.dumps : 딕셔너리를 json 문자열화 시켜준다
    headers=headers
)

response.json()

In [ ]:
data